In [203]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.stats

from matplotlib import animation
from IPython.display import HTML

plt.rcParams['axes.formatter.min_exponent'] = 3

In [204]:
hf= h5py.File('genesis_all.hdf5','r')
print(hf.attrs['info'])

Genesis database


In [205]:
list(hf)

['Gen-HM',
 'Gen-I',
 'Gen-IO',
 'Gen-M-s10',
 'Gen-M-s22',
 'Gen-M-s50',
 'Gen-O',
 'Gen-O-p1',
 'Gen-O-p2',
 'Gen-O-s10',
 'Gen-O-s22',
 'Gen-O-s5',
 'Gen-P',
 'Gen-P-I',
 'Gen-P-IO']

In [206]:
def population_from_hdf5(hfset):
    
    # lists for storing planet porperties
    L_mass= []
    L_sma= []
    L_starID= []
    L_inc= []
    
    for runID, run in hfset.items():    
        
        #print(run['snapshots'].keys())
    
        ID= run['snapshots']['id'][()] # [()] is to extract to numpy arrays
        sma= run['snapshots']['sma'][()] 
        mass= run['snapshots']['mass'][()] 
        time= run['snapshots']['t'][()]
        inc= run['snapshots']['inc'][()]
        
        tgrid= np.unique(time)
        t1= time == tgrid[-1]
            
        #print("--")
        # loop over each planet and add properties to big list
        for planet in run['planets'].values():
            # index to planet in the snapshots
            ip= ID == planet['id'][()] 
                            
            #print(sma[ip&t1][0])
            L_mass.append(mass[ip&t1][0])
            L_sma.append(sma[ip&t1][0]) 
            L_inc.append(inc[ip&t1][0]*(np.pi/180))
            L_starID.append(runID) # identifier for each system
            
            break ## only get innermost planet
        
    starID= np.array(L_starID)
    sma= np.array(L_sma)
    mass= np.array(L_mass)
    inc= np.array(L_inc)
    
    # sort planetary systems inside out
    order= np.lexsort((sma,starID)) 
    
    return dict(starID=starID[order], sma= sma[order], mass=mass[order], inc=inc[order])


In [207]:
popM22 = population_from_hdf5(hf['Gen-M-s22'])


In [216]:
popM22 = population_from_hdf5(hf['Gen-M-s22'])
popHM = population_from_hdf5(hf['Gen-HM'])
popP = population_from_hdf5(hf['Gen-P'])
popO = population_from_hdf5(hf['Gen-O'])
popOp2 = population_from_hdf5(hf['Gen-O-p2'])

In [209]:
len(np.concatenate((popM22['mass'], popHM['mass'], popO['mass'],popOp2['mass'])))

188

In [210]:
masses_to_use = popM22['mass'][popM22['sma'] <= 0.05]

In [211]:
np.savetxt("genesis_masses_sma_masked.txt", masses_to_use)

In [217]:
np.savetxt("genesis_masses_innermost_m22.txt", popM22['mass'])
np.savetxt("genesis_masses_innermost_HM.txt", popHM['mass'])
np.savetxt("genesis_masses_innermost_P.txt", popP['mass'])
np.savetxt("genesis_masses_innermost_O.txt", popO['mass'])
np.savetxt("genesis_masses_innermost_Op2.txt", popOp2['mass'])

In [215]:
np.savetxt("genesis_masses_innermost_popm22.txt", popM22['mass'])